In [127]:
import pandas as pd
import requests
import numpy as np
import os
import json
from dotenv import load_dotenv

In [128]:
load_dotenv()
bearer_token = os.getenv("BEARER_TOKEN")


In [129]:
search_url = "https://api.twitter.com/2/tweets/search/recent"


In [130]:
query_params = {
    'query': 'lang=en',
    'place.fields': 'contained_within,country,country_code,full_name,geo,id,name,place_type',
    'tweet.fields': 'author_id,created_at,geo,id,lang,possibly_sensitive,source,text,withheld',
    'user.fields': 'id,location,name,protected,username,verified,withheld',
    'max_results': 100}
# query_params = {'query': '(from:twitterdev -is:retweet) OR #twitterdev','tweet.fields': 'author_id'}

In [131]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

In [132]:
def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [140]:
def main():
    json_response = connect_to_endpoint(search_url, query_params)
    df = pd.DataFrame.from_dict(json_response['data'])

    

200


Index(['edit_history_tweet_ids', 'source', 'lang', 'text', 'author_id', 'id',
       'possibly_sensitive', 'created_at', 'geo'],
      dtype='object')

In [136]:
if __name__ == "__main__":
    main()

200
